In [1]:
print("Nama   : Aldaffan Sheva G W")
print("Emali  : aldaffan.rodriguez@gmail.com")

Nama   : Aldaffan Sheva G W
Emali  : aldaffan.rodriguez@gmail.com


In [2]:
import csv
import tensorflow as tf
import numpy as np
import urllib

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def dataset(file):
  timestep = []
  sunspots = []

  with open ('/content/drive/MyDrive/Dataset/Sunspots.csv') as file :
    read = csv.reader(file, delimiter = ',')
    next(read)
    for row in read :
      sunspots.append(float(row[2]))
      timestep.append(int(row[0]))
    
  return timestep, sunspots

In [6]:
#print total data
timestep, sunspots = dataset("/content/drive/MyDrive/Dataset/bbc-text.csv")

print(f"{len(timestep)}\n")
print(f"{len(sunspots)}")

3265

3265


In [7]:
def train_window(series, window, batch_size, shuffle_buffer) :
  series = tf.expand_dims(series, axis=-1)
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window + 1, shift=1, drop_remainder=True)
  ds = ds.flat_map(lambda w: w.batch(window + 1))
  ds = ds.shuffle(shuffle_buffer)
  ds = ds.map(lambda w: (w[:-1], w[1:]))
  
  return ds.batch(batch_size).prefetch(1)

In [25]:
timesplit = int(len(timestep) * 0.8)
series = np.array(sunspots)

#Normalisasi
series -= np.min(series)
series /= np.max(series)
t = np.array(timestep)

train_time = t[:timesplit]
x_train = series[:timesplit]
val_time = t[timesplit:]
x_val = series[timesplit:]

train_set = train_window(x_train, window = 30, batch_size = 32, 
                         shuffle_buffer = 1000)
valid_set = train_window(x_val, window = 30, batch_size = 32, 
                         shuffle_buffer = 10)

In [28]:
print(f"{len(train_time)} Total Time For Training\n")
print(f"{len(x_train)} Total Sunspots For Training\n")
print(f"{len(val_time)} Total Time For Validation\n")
print(f"{len(x_val)} Total Sunspots For Validation")

2612 Total Time For Training

2612 Total Sunspots For Training

653 Total Time For Validation

653 Total Sunspots For Validation


In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters = 32,
                           kernel_size = 5,
                           strides = 1,
                           padding = 'causal',
                           activation = 'relu',
                           input_shape = [None, 1]),
    tf.keras.layers.LSTM(32, return_sequences = True),
    tf.keras.layers.Dense(30, activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'relu'),
    tf.keras.layers.Dense(1)
    ])

model.compile(loss = 'mae',
              optimizer = 'adam',
              metrics = ['mae'])
model.fit(train_set, epochs = 10, validation_data = valid_set)

Epoch 1/10
81/81 [==============================] - 5s 31ms/step - loss: 0.0826 - mae: 0.0826 - val_loss: 0.0512 - val_mae: 0.0512
Epoch 2/10
81/81 [==============================] - 2s 22ms/step - loss: 0.0502 - mae: 0.0502 - val_loss: 0.0458 - val_mae: 0.0458
Epoch 3/10
81/81 [==============================] - 2s 22ms/step - loss: 0.0478 - mae: 0.0478 - val_loss: 0.0445 - val_mae: 0.0445
Epoch 4/10
81/81 [==============================] - 2s 23ms/step - loss: 0.0466 - mae: 0.0466 - val_loss: 0.0430 - val_mae: 0.0430
Epoch 5/10
81/81 [==============================] - 2s 23ms/step - loss: 0.0456 - mae: 0.0456 - val_loss: 0.0425 - val_mae: 0.0425
Epoch 6/10
81/81 [==============================] - 2s 22ms/step - loss: 0.0454 - mae: 0.0454 - val_loss: 0.0421 - val_mae: 0.0421
Epoch 7/10
81/81 [==============================] - 2s 22ms/step - loss: 0.0450 - mae: 0.0450 - val_loss: 0.0421 - val_mae: 0.0421
Epoch 8/10
81/81 [==============================] - 2s 23ms/step - loss: 0.0450 - m